# 漫画 Recommend System

レビュー類似度を用いた 漫画 Recommend System の開発

使用する映画レビューサイトは　http://www.manngareview.com/

### 構成

1. スクレイピング/SQL
2. レビューの形態素解析
3. 特徴量行列の作成
4. モデル作成

### 準備

In [10]:
import pandas as pd
from pandas import Series,DataFrame

In [ ]:
import sqlite3
con = sqlite3.connect("movies.db")

In [ ]:
# 関数を作って、SQL文の実行結果をDataFrameにして返します。
def sql_to_df(sql_query):

    # read_sqlの引数に、SQL文とデータベースへのConnectionを渡します。
    df = pd.read_sql(sql_query, con)

    # 結果のDataFrameを返します。
    return df

### 1. スクレイピング/SQL

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
import time
import re

In [8]:
i = 0

i_id = []
all_title = []
data = []

for num in range(42,45):
    time.sleep(2)

    url = 'http://www.manngareview.com/mannga/' + str(num)
    
    result = requests.get(url)
    c = result.content

    soup = BeautifulSoup(c, "lxml")
    
    title = soup.find("title")
    title_text = title.find(text=True)
    print(title_text)
    
    #for a in soup.find_all('a'):
    #pa = a.get('href')
    #if re.match('.*page:[^1]', pa):
        #pa = url + pa
        #if pa not in c_url:
            #c_url.append(pa)
            
    #c_url.insert(0, url)

    # テーブルから行をすべて探し出します。
    reviews = soup.find_all('p',{'class':'review_string'})
    
    for item in reviews:
        text = item.find_all(text=True)
        print(text)
        
        i += 1
        
        i_id.append(i)
        all_title.append(title_text)            
        data.append(text)  

「愛しのアイリーン」のレビュー / 漫画レビュー.com
['魂が削られる作品。', '\r\n田舎の山奥に両親と住むパチンコ屋勤めの42歳の主人公岩男は妻も彼女もいないパッとしない人生を歩んでいて、職場でのほのかな恋愛をキッカケに嫁を求めてフィリピンに見合いに飛び立つ。そして連れて帰ってきた若いフィリピーナ、アイリーンとの間にはもちろん愛なんてなく…。', '\n', '\r\n約20年前の作品だけど今でもこんな世界ありますよね。', '\r\n現実にありそうな設定をデフォルメされた強烈なキャラクター達によってすごい展開になっていく、一度読んだだけでも忘れられない物語。', '\r\nやっぱすごいわ新井英樹。', '\r\nそれぞれのエゴがどんどんぶつかって、皆が不幸になっていき読んでて胸が痛くなるけど引き込まれて最後まで一気に読んでしまいました。', '\n', '\n']
['できれば漫画でも見たくない展開。', '\r\n人生で一番自分が避けてきた展開。', '\r\n読めない展開。', '\n', '\r\nのオンパレードでね??笑うしかありませんでした', '\r\nオーバーすぎる描写だけど、岩男の家みたいな家族、結構いそうだし。', '\r\n絶対関わり合いになりたくないけどｗｗｗ', '\n', '\r\nでもラストが好きです。女ってたくましいっていう真実。', '\r\nそれが現実。', '\n', '\n']
['愛という言葉を借りて、人間のダークサイドを描き出した欲望丸出し漫画。', '\n', '\r\nもうなんかね、とりあえず汚い。えげつない。勢いだけ。', '\r\n読んでいてとにかく疲れる。心地悪い。', '\n', '\r\nしかし、読み続けてしまう。のめり込んでしまう。', '\r\nこれが、新井英樹の力。もはや麻薬。計算というよりは暴走で描かれた故の愛おしさ。', '\n', '\r\n不必要な性描写は、同人誌か、と思えるほど。', '\r\nしかも、キャラクターがみんな不細工なのは新井作品の共通点。', '\r\nかわいい萌えキャラなんて描いているけど、鏡を見てみなよ、現実のお前の顔なんて所詮こんなもんだよ、と言わんがばかり。', '\r\n汚さを描き続けることで、かえって、その中に光る「愛」の美しさが強調されるというコントラス

In [11]:
i_id =Series(i_id)
all_title = Series(all_title)
data = Series(data)

In [12]:
# 連結してDataFrameにします。
reviews_df = pd.concat([i_id, all_title, data],axis=1)
reviews_df.columns=['id', 'title','review']

In [13]:
reviews_df

,id,title,review
0,1,「愛しのアイリーン」のレビュー / 漫画レビュー.com,"[魂が削られる作品。, 田舎の山奥に両親と住むパチンコ屋勤めの42歳の主人公岩男は妻も彼..."
1,2,「愛しのアイリーン」のレビュー / 漫画レビュー.com,"[できれば漫画でも見たくない展開。, 人生で一番自分が避けてきた展開。, 読めない展..."
2,3,「愛しのアイリーン」のレビュー / 漫画レビュー.com,"[愛という言葉を借りて、人間のダークサイドを描き出した欲望丸出し漫画。, , もうなん..."
3,4,「愛しのアイリーン」のレビュー / 漫画レビュー.com,"[このままでは一生結婚できなさそうなどド田舎の農家の息子と, その男に金で買われて結婚す..."
4,5,「愛しのアイリーン」のレビュー / 漫画レビュー.com,"[新井漫画の中でも最も「性」を前面に押し出した作品, ドギツイなんてもんじゃない, , ..."
5,6,「ジョジョの奇妙な冒険」のレビュー / 漫画レビュー.com,"[好き嫌いの分かれる絵柄だが、, この絵はこの漫画の一番の売りだと思う。, 最初は北..."
6,7,「ジョジョの奇妙な冒険」のレビュー / 漫画レビュー.com,"[めちゃくちゃ好きな作品です。, しかし、全体でならすと、意外にも７点でした。。。, ..."
7,8,「ジョジョの奇妙な冒険」のレビュー / 漫画レビュー.com,"[面白い。, , 一部だけ読んで切るのは勿体ない作品。, もちろんこの作品の魅力の..."
8,9,「ジョジョの奇妙な冒険」のレビュー / 漫画レビュー.com,"[いくらなんでもみんなの評価は高すぎ。, ジャンプ連載中はいつも後ろの方だったし、事実つ..."
9,10,「ジョジョの奇妙な冒険」のレビュー / 漫画レビュー.com,"[「ザ・ワールド」で、モノクロームな背景で時が止まった表現、, キングクリムゾンで、背景..."


In [21]:
reviews_df.count()

id        15
title     15
review    15
dtype: int64

In [7]:
i = 0

i_id = []
all_title = []
data = []

for num in range(400,450):
    c_url = []
    
    time.sleep(2)

    url = 'http://www.manngareview.com/mannga/' + str(num)
    
    result = requests.get(url)
    c = result.content

    soup = BeautifulSoup(c, "lxml")
    
    title = soup.find("title")
    title_text = title.find(text=True)

    #for a in soup.find_all('a'):
        #pa = a.get('href')
        #if re.match('.*page:[^1]', pa):
            #pa = url + pa
            #if pa not in c_url:
                #c_url.append(pa)
    
    #c_url.insert(0, url)

    for item in c_url:
        
        time.sleep(2)
        
        print(item)
        
        result = requests.get(item)
        c = result.content

        soup = BeautifulSoup(c, "lxml")
        
        #for a in soup.find_all('a'):
        #pa = a.get('href')
        #if re.match('.*page:[^1]', pa):
            #pa = url + pa
            #if pa not in c_url:
                #cc_url.append(pa)

        reviews = soup.find_all('p', {'class':'review_string'})
        text = reviews.find_all(text=True)
        data_rows_renketsu = ""
        data_rows_renketsu += text

        i += 1    
                
        print("i_id : ", i)
        print("title_text : ", title_text)                
        print("data_rows_renketsu : ", data_rows_renketsu)
            
        i_id.append(i)
        all_title.append(title_text)            
        data.append(data_rows_renketsu)  

データはSQLで扱いやすいように、DataFrame型に変えていきます。

・data = [] / list

・data = Series(data)

・legislative_df = pd.concat([data, axis=1) / DataFrame

In [9]:
i_id =Series(i_id)
all_title = Series(all_title)
data = Series(data)

In [10]:
# 連結してDataFrameにします。
reviews_df = pd.concat([i_id, all_title, data],axis=1)
reviews_df.columns=['id', 'title','review']

In [11]:
reviews_df

,id,title,review
0,1,愛は静けさの中に - みんなのシネマレビュー,35．《ネタバレ》 ヒロインのマーリー・マトリンは、イザベル・アジャーニみたいに美人だけど、...
1,2,愛は静けさの中に - みんなのシネマレビュー,34．昔ながらのよくある正統派のラブストーリーだが、聾学校という舞台背景が設定の妙味に。ヒロ...
2,3,愛は静けさの中に - みんなのシネマレビュー,33．耳が聞こえる男と耳が聞こえない女は、どうすれば一つの同じ世界に生きることができるだろう...
3,4,愛は静けさの中に - みんなのシネマレビュー,32．生徒や恋人、聾唖者との関わりを重視したドラマだと思っていたが、よくある一組のカップルの...
4,5,愛は静けさの中に - みんなのシネマレビュー,31．《ネタバレ》 二人の恋のゆくえより、生徒とのやりとりのほうが面白かった。生徒達はかわい...
5,6,愛は静けさの中に - みんなのシネマレビュー,30．《ネタバレ》 マーリー・マトリン美しい… 賞受賞も納得の内容で演出面も素晴らしい！ ...
6,7,愛は静けさの中に - みんなのシネマレビュー,29．教師役として抑えた演技が光るウィリアム・ハートと、この作品でオスカーを獲得した女優マー...
7,8,愛は静けさの中に - みんなのシネマレビュー,28．自分が好きだった人と、ヒロインが同じ髪型（「フラッシュダンス」のジェニファービールスと...
8,9,愛は静けさの中に - みんなのシネマレビュー,27．こんな純愛ドラマがあったなぁとか思いながら鑑賞しました。ストーリーには凝ったところもな...
9,10,愛は静けさの中に - みんなのシネマレビュー,26．ウィリアム・ハートとマーリー・マトリン、この二人の演技がすごい。特にマーリー・マトリン...


In [12]:
hiku_df = reviews_df.ix[reviews_df['review'].str.contains('"')]

In [13]:
hiku_df

,id,title,review
21,22,愛は静けさの中に - みんなのシネマレビュー,14． 原作はマーク・メドフの舞台劇「小さき神の子等」。ヒロインのマーリー・マトリンは舞台か...


In [14]:
reviews_df = reviews_df.drop([21])

In [15]:
reviews_df.ix[reviews_df['review'].str.contains('"')]

,id,title,review


##### 事前に作成しているテーブル

CREATE TABLE reviews(
    id text autoincrement primary key,
    title char(30),
    review char(30)
 );

In [104]:
sql_query = '''
SELECT * FROM reviews
'''
res_df = sql_to_df(sql_query)

In [105]:
res_df

,id,title,review


In [15]:
con.execute("delete from reviews")

In [106]:
for item in range(len(reviews_df)):
    
    time.sleep(1)
    
    i_id = reviews_df.iloc[item,0]
    title = reviews_df.iloc[item,1]
    review = reviews_df.iloc[item,2]
    
    #print(title)
    #print(review)
    
    sql_query = '''
    INSERT INTO reviews VALUES ({i}, "{title}", "{review}")
    '''.format(i=i_id, title=title, review=review)

    con.execute(sql_query)
    print(sql_query)



    INSERT INTO reviews VALUES (1, "愛は静けさの中に - みんなのシネマレビュー", "35．《ネタバレ》　ヒロインのマーリー・マトリンは、イザベル・アジャーニみたいに美人だけど、ただ美人だけというきっかけで恋に落ちる主人公に感情移入はできない。二人の心が近づいていく過程も無理やり。ヒロインは女子高生じゃないけど、こういう安っぽい映画で勘違いして、ストーカーや猥褻行為をする教師も出てくるのではないか。全てがハリウッドらしい、表面的で、無性に苛立った映画だった。 【クロエ】さん [CS・衛星(字幕)] 4点（2014-09-09 22:40:19）")
    

    INSERT INTO reviews VALUES (2, "愛は静けさの中に - みんなのシネマレビュー", "34．昔ながらのよくある正統派のラブストーリーだが、聾学校という舞台背景が設定の妙味に。ヒロインもろうあ者だということで、彼女ありきで作られた脚本かと思ったら、もともとが原作のある舞台劇だと知ってちょっとびっくりした。映像演出音楽等はいかにも８０年代。ヒロインのセリフがひと言もないので、さすがに後半からは重苦しさを感じてしまったけど、プールのシーンは印象に残っている。 【MAHITO】さん [ビデオ(字幕)] 4点（2012-11-28 01:32:53）")
    

    INSERT INTO reviews VALUES (3, "愛は静けさの中に - みんなのシネマレビュー", "33．耳が聞こえる男と耳が聞こえない女は、どうすれば一つの同じ世界に生きることができるだろうか。これは純愛ドラマだけどひとつの社会ドラマでもある。作品賞を初め主演男優、主演女優、助演女優などアカデミー賞にノミネートされた結果、マーリー・マトリンだけがその栄冠を射止めた。なるほどと思わせる映画だ。彼女は美しいだけでなく聾唖者、しかし聾唖者に関係なくその演技が認められたのだと思う。出会って最初の会話「僕が話し方を教えるよ」「私はあなたに掃除の仕方を教える」から参った。 【ESPERANZA】さん [DVD(字幕)] 8点（2012-08-18 22:46:03）")
    

    INSERT INTO reviews VALUES (4, "愛は静けさの

OperationalError: near "See": syntax error

In [107]:
sql_query = '''
SELECT * FROM reviews
'''
res_df = sql_to_df(sql_query)

In [108]:
res_df

,id,title,review
0,1,愛は静けさの中に - みんなのシネマレビュー,35．《ネタバレ》 ヒロインのマーリー・マトリンは、イザベル・アジャーニみたいに美人だけど、...
1,2,愛は静けさの中に - みんなのシネマレビュー,34．昔ながらのよくある正統派のラブストーリーだが、聾学校という舞台背景が設定の妙味に。ヒロ...
2,3,愛は静けさの中に - みんなのシネマレビュー,33．耳が聞こえる男と耳が聞こえない女は、どうすれば一つの同じ世界に生きることができるだろう...
3,4,愛は静けさの中に - みんなのシネマレビュー,32．生徒や恋人、聾唖者との関わりを重視したドラマだと思っていたが、よくある一組のカップルの...
4,5,愛は静けさの中に - みんなのシネマレビュー,31．《ネタバレ》 二人の恋のゆくえより、生徒とのやりとりのほうが面白かった。生徒達はかわい...
5,6,愛は静けさの中に - みんなのシネマレビュー,30．《ネタバレ》 マーリー・マトリン美しい… 賞受賞も納得の内容で演出面も素晴らしい！ ...
6,7,愛は静けさの中に - みんなのシネマレビュー,29．教師役として抑えた演技が光るウィリアム・ハートと、この作品でオスカーを獲得した女優マー...
7,8,愛は静けさの中に - みんなのシネマレビュー,28．自分が好きだった人と、ヒロインが同じ髪型（「フラッシュダンス」のジェニファービールスと...
8,9,愛は静けさの中に - みんなのシネマレビュー,27．こんな純愛ドラマがあったなぁとか思いながら鑑賞しました。ストーリーには凝ったところもな...
9,10,愛は静けさの中に - みんなのシネマレビュー,26．ウィリアム・ハートとマーリー・マトリン、この二人の演技がすごい。特にマーリー・マトリン...


In [92]:
#con.close()

ここまでで、レビューのデータベースへの格納が終わり。

次はレビューを読みだして形態素解析を行っていく。

### 2. レビューの形態素解析

In [16]:
import MeCab

In [110]:
sql_query = '''
SELECT * FROM reviews
'''
reviews_df = sql_to_df(sql_query)

In [17]:
mecab = MeCab.Tagger ('-d /usr/local/lib/mecab/dic/mecab-ipadic-neologd')

##### 事前に作成しているテーブル

CREATE TABLE cnt_table(
    id int,
    title char(50),
    word char(30),
    num int
);

In [236]:
for item in range(len(reviews_df)):
    time.sleep(0.1)    
    i_id = reviews_df.iloc[item, 0]
    title = reviews_df.iloc[item,1]
    review = reviews_df.iloc[item,2]
    print(i_id)
    print(title)
    print(review)
    
    text = review
    mecab.parse('')#文字列がGCされるのを防ぐ
    node = mecab.parseToNode(text)
    while node:
        #単語を取得
        word = node.surface
        #品詞を取得
        pos = node.feature.split(",")[1]
        print('{}'.format(word))
        #print('{0} , {1}'.format(word, pos))
        
        #次の単語に進める
        node = node.next
        
        sql_query = '''
        INSERT INTO cnt_table(id, title, word, num) values({i_id}, "{title}", "{word}", 1)
        '''.format(i_id=i_id, title=title, word=word)
        
        con.execute(sql_query)
        

1
愛は静けさの中に - みんなのシネマレビュー
35．《ネタバレ》　ヒロインのマーリー・マトリンは、イザベル・アジャーニみたいに美人だけど、ただ美人だけというきっかけで恋に落ちる主人公に感情移入はできない。二人の心が近づいていく過程も無理やり。ヒロインは女子高生じゃないけど、こういう安っぽい映画で勘違いして、ストーカーや猥褻行為をする教師も出てくるのではないか。全てがハリウッドらしい、表面的で、無性に苛立った映画だった。 【クロエ】さん [CS・衛星(字幕)] 4点（2014-09-09 22:40:19）

35
．
《
ネタバレ
》
　
ヒロイン
の
マーリー・マトリン
は
、
イザベル・アジャーニ
みたい
に
美人
だ
けど
、
ただ
美人
だけ
という
きっかけ
で
恋
に
落ちる
主人公
に
感情移入
は
でき
ない
。
二
人
の
心
が
近づい
て
いく
過程
も
無理やり
。
ヒロイン
は
女子高生
じゃ
ない
けど
、
こういう
安っぽい
映画
で
勘違い
し
て
、
ストーカー
や
猥褻
行為
を
する
教師
も
出
て
くる
の
で
は
ない
か
。
全て
が
ハリウッド
らしい
、
表面
的
で
、
無性に
苛立っ
た
映画
だっ
た
。
【
クロエ
】
さん
[
CS
・
衛星
(
字幕
)]
4
点
（
2014
-
09
-
09
22
:
40
:
19
）

2
愛は静けさの中に - みんなのシネマレビュー
34．昔ながらのよくある正統派のラブストーリーだが、聾学校という舞台背景が設定の妙味に。ヒロインもろうあ者だということで、彼女ありきで作られた脚本かと思ったら、もともとが原作のある舞台劇だと知ってちょっとびっくりした。映像演出音楽等はいかにも８０年代。ヒロインのセリフがひと言もないので、さすがに後半からは重苦しさを感じてしまったけど、プールのシーンは印象に残っている。 【MAHITO】さん [ビデオ(字幕)] 4点（2012-11-28 01:32:53）

34
．
昔ながらの
よく
ある
正統派
の
ラブストーリー
だ
が
、
聾学校
という
舞台
背景
が
設定
の
妙味
に
。
ヒロイン
も
ろうあ
者
だ
という
こと
で
、
彼女
あり
き
で
作ら
れ
た
脚本
か
と
思っ
たら
、
もともと
が
原作
の
あ

In [137]:
sql_query = '''
SELECT * FROM cnt_table
'''
words_df = sql_to_df(sql_query)

In [138]:
words_df.tail()

,id,title,word,num


In [ ]:
#con.close()

### 3. 特徴量行列の作成

In [237]:
i = 1
words_cnt = []

In [238]:
#The first step —————————————————————————————————————————————————
#全映画で使われたwordをとりだす。
sql_query = '''
SELECT word FROM cnt_table WHERE word not like '%''%' GROUP BY word ORDER BY sum(num) desc
'''
words_used_df = sql_to_df(sql_query)

In [239]:
words_used_df.tail(50)

,word
4821,高級
4822,高騰
4823,髪型
4824,魔界
4825,鱗
4826,鳩
4827,鳴か
4828,鳴り響い
4829,鵜呑み
4830,麗人


In [289]:
#The second step ————————————————————————————————————————————————
num = 0
query_1 = '''SELECT id, '''
for item in range(len(words_used_df)):
    #レビューIDごとに出現しているwordでカウントする。
    query_1 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
    num += 1
    
    if num==1900:
        break
    
query_1 += '''title FROM cnt_table GROUP BY id, title'''

In [248]:
#The second step ————————————————————————————————————————————————
num = 0
query_2 = '''SELECT id, '''
for item in range(len(words_used_df)):
    num += 1
    if num >= 1900:
        #レビューIDごとに出現しているwordでカウントする。
        query_2 += '''SUM(CASE WHEN word = '{word}' THEN num ELSE 0 END) AS a{num}, '''.format(word=words_used_df.iloc[item, 0], num=num)
        
        if num==3800:
            break
    
query_2 += '''title FROM cnt_table GROUP BY id, title'''

In [302]:
res_df_1 = sql_to_df(query_1)

In [303]:
res_df_1.tail()

,id,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a1891,a1892,a1893,a1894,a1895,a1896,a1897,a1898,a1899,title
467,469,4,1,1,0,2,2,1,1,2,...,0,0,0,0,0,0,0,0,0,あなたに降る夢 - みんなのシネマレビュー
468,470,0,0,0,0,2,2,0,0,2,...,0,0,0,0,0,0,0,0,0,あなたに降る夢 - みんなのシネマレビュー
469,471,6,1,2,3,2,2,6,2,2,...,0,0,0,0,0,0,0,0,0,あなたに降る夢 - みんなのシネマレビュー
470,472,3,1,2,2,2,2,3,1,2,...,0,0,0,0,0,0,0,0,0,あなたに降る夢 - みんなのシネマレビュー
471,473,0,3,2,1,2,2,0,0,2,...,0,0,0,0,0,0,0,0,0,あなたに降る夢 - みんなのシネマレビュー


In [323]:
res_df_1 = res_df_1.drop('title', axis=1)

In [249]:
res_df_2 = sql_to_df(query_2)

In [250]:
res_df_2

,id,a1900,a1901,a1902,a1903,a1904,a1905,a1906,a1907,a1908,...,a3792,a3793,a3794,a3795,a3796,a3797,a3798,a3799,a3800,title
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
1,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
2,3,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
4,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
5,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
6,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
7,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
8,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
9,10,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー


In [38]:
#con.close()

In [324]:
res_df = pd.concat([res_df_1,res_df_2], axis=1)

In [325]:
res_df

,id,a0,a1,a2,a3,a4,a5,a6,a7,a8,...,a3792,a3793,a3794,a3795,a3796,a3797,a3798,a3799,a3800,title
0,1,4,3,6,2,2,2,2,3,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
1,2,4,5,5,4,2,2,3,3,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
2,3,6,5,5,6,2,2,4,1,2,...,0,1,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
3,4,1,3,2,1,2,2,3,1,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
4,5,5,4,2,3,2,2,5,3,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
5,6,0,3,1,3,2,2,1,1,2,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
6,7,3,3,3,3,2,2,3,1,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
7,8,2,3,2,3,2,2,4,1,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
8,9,3,3,1,3,2,2,4,0,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー
9,10,3,3,1,2,2,2,0,2,0,...,0,0,0,0,0,0,0,0,0,愛は静けさの中に - みんなのシネマレビュー


### 4. モデル作成

ここでは、UdemyのPython実践講座の Lec80-81 を参考にしてロジスティック回帰での多値分類を行う。

In [326]:
import numpy as np
import pandas as pd
from pandas import Series,DataFrame

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')

%matplotlib inline

In [327]:
from sklearn import linear_model

参考 : http://qiita.com/yshi12/items/9f3f8ae69588da9f018f

In [328]:
res_df.target = res_df['title']

In [329]:
res_numpyMatrix_target = res_df.target.as_matrix()

In [51]:
res_numpyMatrix_target

array(['愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー

In [52]:
res_df.target

0      愛は静けさの中に - みんなのシネマレビュー
1      愛は静けさの中に - みんなのシネマレビュー
2      愛は静けさの中に - みんなのシネマレビュー
3      愛は静けさの中に - みんなのシネマレビュー
4      愛は静けさの中に - みんなのシネマレビュー
5      愛は静けさの中に - みんなのシネマレビュー
6      愛は静けさの中に - みんなのシネマレビュー
7      愛は静けさの中に - みんなのシネマレビュー
8      愛は静けさの中に - みんなのシネマレビュー
9      愛は静けさの中に - みんなのシネマレビュー
10     愛は静けさの中に - みんなのシネマレビュー
11     愛は静けさの中に - みんなのシネマレビュー
12     愛は静けさの中に - みんなのシネマレビュー
13     愛は静けさの中に - みんなのシネマレビュー
14     愛は静けさの中に - みんなのシネマレビュー
15     愛は静けさの中に - みんなのシネマレビュー
16     愛は静けさの中に - みんなのシネマレビュー
17     愛は静けさの中に - みんなのシネマレビュー
18     愛は静けさの中に - みんなのシネマレビュー
19     愛は静けさの中に - みんなのシネマレビュー
20     愛は静けさの中に - みんなのシネマレビュー
21     愛は静けさの中に - みんなのシネマレビュー
22     愛は静けさの中に - みんなのシネマレビュー
23     愛は静けさの中に - みんなのシネマレビュー
24     愛は静けさの中に - みんなのシネマレビュー
25     愛は静けさの中に - みんなのシネマレビュー
26     愛は静けさの中に - みんなのシネマレビュー
27     愛は静けさの中に - みんなのシネマレビュー
28     愛は静けさの中に - みんなのシネマレビュー
29     愛は静けさの中に - みんなのシネマレビュー
                ...          
935       アポロ１３ - みんなのシネマレビュー
936       アポロ１３ - みんなのシネマレビュー
937       

In [330]:
res_df.data = res_df.iloc[ : , 1:3800]

In [331]:
res_numpyMatrix_data = res_df.iloc[ : , 1:3800].as_matrix()

In [55]:
res_numpyMatrix_data

array([[8, 4, 4, ..., 0, 0, 0],
       [8, 0, 4, ..., 0, 0, 0],
       [8, 0, 4, ..., 0, 0, 0],
       ..., 
       [2, 0, 1, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0]])

In [56]:
res_df.data

,a0,a1,a2,a3,a4,a5,a6,a7,a8,a9,...,a1489,a1490,a1491,a1492,a1493,a1494,a1495,a1496,a1497,a1498
0,8,4,4,4,4,4,4,8,12,4,...,0,0,0,0,0,0,0,0,0,0
1,8,0,4,0,0,0,0,8,20,0,...,0,0,0,0,0,0,0,0,0,0
2,8,0,4,0,0,0,0,0,20,4,...,0,0,0,0,0,0,0,0,0,0
3,8,0,4,0,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,4,4,4,4,4,0,16,0,...,0,0,0,0,0,0,0,0,0,0
5,8,0,4,4,4,4,28,0,12,4,...,0,0,0,0,0,0,0,0,0,0
6,8,0,4,0,0,0,0,0,12,4,...,0,0,0,0,0,0,0,0,0,0
7,8,0,4,0,0,0,0,4,12,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,4,0,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0
9,8,0,4,0,0,0,0,0,12,8,...,0,0,0,0,0,0,0,0,0,0


In [332]:
# 説明変数をXに
X = res_numpyMatrix_data

#目的変数をYに
Y = res_numpyMatrix_target

In [333]:
#pandas.DataFrameにしておきましょう。

_data = DataFrame(X)

_target = DataFrame(Y,columns=['title'])

In [60]:
#実行しなくて良い
_data

,0,1,2,3,4,5,6,7,8,9,...,1489,1490,1491,1492,1493,1494,1495,1496,1497,1498
0,8,4,4,4,4,4,4,8,12,4,...,0,0,0,0,0,0,0,0,0,0
1,8,0,4,0,0,0,0,8,20,0,...,0,0,0,0,0,0,0,0,0,0
2,8,0,4,0,0,0,0,0,20,4,...,0,0,0,0,0,0,0,0,0,0
3,8,0,4,0,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,4,4,4,4,4,0,16,0,...,0,0,0,0,0,0,0,0,0,0
5,8,0,4,4,4,4,28,0,12,4,...,0,0,0,0,0,0,0,0,0,0
6,8,0,4,0,0,0,0,0,12,4,...,0,0,0,0,0,0,0,0,0,0
7,8,0,4,0,0,0,0,4,12,0,...,0,0,0,0,0,0,0,0,0,0
8,8,0,4,0,0,0,0,0,12,0,...,0,0,0,0,0,0,0,0,0,0
9,8,0,4,0,0,0,0,0,12,8,...,0,0,0,0,0,0,0,0,0,0


In [61]:
#実行しなくて良い
_target

,title
0,愛は静けさの中に - みんなのシネマレビュー
1,愛は静けさの中に - みんなのシネマレビュー
2,愛は静けさの中に - みんなのシネマレビュー
3,愛は静けさの中に - みんなのシネマレビュー
4,愛は静けさの中に - みんなのシネマレビュー
5,愛は静けさの中に - みんなのシネマレビュー
6,愛は静けさの中に - みんなのシネマレビュー
7,愛は静けさの中に - みんなのシネマレビュー
8,愛は静けさの中に - みんなのシネマレビュー
9,愛は静けさの中に - みんなのシネマレビュー


scikit-learnを使った多クラス分類


In [334]:
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split

logreg = LogisticRegression()

# データを分割します。テストが全体の40%になるようにします。
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.4,random_state=3)

# データを使って学習します。
logreg.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [173]:
#実行しなくて良い
X_train

array([[2, 0, 1, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       ..., 
       [2, 0, 1, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0],
       [2, 0, 1, ..., 0, 0, 0]])

In [174]:
#実行しなくて良い
Y_train

array(['あなたに降る夢 - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュー',
       'あなたに降る夢 - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アサシン

In [335]:
# 精度を計算するのに便利なツールです。
from sklearn import metrics

# テストデータを予測します。
Y_pred = logreg.predict(X_test)

# 精度を計算してみましょう。
print(metrics.accuracy_score(Y_test,Y_pred))

0.830687830688


In [86]:
Y_test

array(['あなたに降る夢 - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', 'アトランティック・シティ - みんなのシネマレビュー',
       'アグネス - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       'あなたに降る夢 - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       '愛は静けさの中に - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'あなたに降る夢 - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュ

In [87]:
Y_pred

array(['愛は静けさの中に - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'あなたに降る夢 - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'あなたに降る夢 - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュー',
       'あなたに降る夢 - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', '愛は静けさの中に - みんなのシネマレビュー',
       'アサシン(1993) - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アダムス・ファミリー - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'アサシン(1993) - みんなのシネマレビュー',
       'アダムス・ファミリー２ - みんなのシネマレビュー', 'アダムス・ファミリー２ - みんなのシネマレビュー',
       'あなたに降る夢 - みんなのシネマレビュー', 'アダムス・ファミリー - みんなのシネマレビュ

In [336]:
Y_test = Series(Y_test)
Y_pred = Series(Y_pred)

In [337]:
hikaku = []

In [338]:
hikaku = pd.concat([Y_test, Y_pred], axis=1)

In [339]:
hikaku

,0,1
0,愛は静けさの中に - みんなのシネマレビュー,愛は静けさの中に - みんなのシネマレビュー
1,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
2,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
3,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
4,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー - みんなのシネマレビュー
5,あなたに降る夢 - みんなのシネマレビュー,あなたに降る夢 - みんなのシネマレビュー
6,アダムス・ファミリー - みんなのシネマレビュー,アダムス・ファミリー - みんなのシネマレビュー
7,アダムス・ファミリー２ - みんなのシネマレビュー,アダムス・ファミリー２ - みんなのシネマレビュー
8,あなたに降る夢 - みんなのシネマレビュー,あなたに降る夢 - みんなのシネマレビュー
9,アサシン(1993) - みんなのシネマレビュー,アサシン(1993) - みんなのシネマレビュー
